In [2]:
import sys
import math
import pdb
state_M = 4
word_N = 0
 
A_dic = {}
B_dic = {}
Count_dic = {}
Pi_dic = {}
word_set = set()
state_list = ['B','M','E','S']
line_num = -1
 
INPUT_DATA = "datas/trainCorpus.txt_utf8"
PROB_START = "prob_start.py"   #初始状态概率
PROB_EMIT = "prob_emit.py"     #发射概率
PROB_TRANS = "prob_trans.py"   #转移概率
 

def init():
    global state_M
    global word_N
    for state in state_list:
        A_dic[state] = {}
        for state1 in state_list:
            A_dic[state][state1] = 0.0
    for state in state_list:
        Pi_dic[state] = 0.0
        B_dic[state] = {}
        Count_dic[state] = 0

# 输入词语  输出状态 B代表其实字，M代表中间字，E代表结束字，S代表单字成词
def getList(input_str):
    outpout_str = []
    if len(input_str) == 1:
        outpout_str.append('S')
    elif len(input_str) == 2:
        outpout_str = ['B','E']
    else:
        M_num = len(input_str) -2
        M_list = ['M'] * M_num
        outpout_str.append('B')
        outpout_str.extend(M_list)
        outpout_str.append('S')
    return outpout_str
 
# 输出模型的三个参数：初始概率+转移概率+发射概率
def Output():
    start_fp = open(PROB_START,'w',encoding='utf-8')
    emit_fp = open(PROB_EMIT,'w',encoding='utf-8')
    trans_fp = open(PROB_TRANS,'w',encoding='utf-8')
     
    for key in Pi_dic:
        Pi_dic[key] = Pi_dic[key] * 1.0 / line_num
    start_fp.write(str(Pi_dic))
 
    for key in A_dic:
        for key1 in A_dic[key]:
            A_dic[key][key1] = A_dic[key][key1] / Count_dic[key]
    trans_fp.write(str(A_dic))
 
    for key in B_dic:
        for word in B_dic[key]:
            B_dic[key][word] = B_dic[key][word] / Count_dic[key]
    emit_fp.write(str(B_dic))
    
 
    start_fp.close()
    emit_fp.close()
    trans_fp.close()
 
 
def main():
    ifp = open(INPUT_DATA,encoding='utf-8')
    init()
    global word_set
    global line_num
    for line in ifp:
        line_num += 1
        if line_num % 10000 == 0:
            print (line_num)
 
        line = line.strip()
        if not line:
            continue
        # line = line.decode("utf-8","ignore")
 
        word_list = []
        for i in range(len(line)):
            if line[i] == " ":continue
            word_list.append(line[i])
        word_set = word_set | set(word_list)
 
 
        lineArr = line.split(" ")
        line_state = []
        for item in lineArr:
            # 一句话对应一行连续的状态
            line_state.extend(getList(item))
 
        if len(word_list) != len(line_state):
            print (sys.stderr,"[line_num = %d][line = %s]" % (line_num, line.encode("utf-8",'ignore')))
        else:
            for i in range(len(line_state)):
                if i == 0:
                    # 计算转移概率
                    Pi_dic[line_state[0]] += 1
                    Count_dic[line_state[0]] += 1
                else:
                    # 计算发射概率
                    A_dic[line_state[i-1]][line_state[i]] += 1
                    Count_dic[line_state[i]] += 1
                    # if not B_dic[line_state[i]].has_key(word_list[i]):
                    if word_list[i] not in B_dic[line_state[i]]:
                        B_dic[line_state[i]][word_list[i]] = 0.0
                    else:
                        B_dic[line_state[i]][word_list[i]] += 1
    Output()
    ifp.close()
    
if __name__ == "__main__":
    main()


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000


In [1]:
import os
import sys
import pdb

def load_model(f_name):
    ifp = open(f_name, 'rb')
    return eval(ifp.read())
 
prob_start = load_model("prob_start.py")
prob_trans = load_model("prob_trans.py")
prob_emit = load_model("prob_emit.py")
 
def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}] #tabular
    path = {}
    for y in states: #init
        V[0][y] = start_p[y] * emit_p[y].get(obs[0],0)
        path[y] = [y]
    for t in range(1,len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            (prob,state ) = max([(V[t-1][y0] * trans_p[y0].get(y,0) * emit_p[y].get(obs[t],0) ,y0) for y0 in states if V[t-1][y0]>0])
            V[t][y] =prob
            newpath[y] = path[state] + [y]
        path = newpath
    (prob, state) = max([(V[len(obs) - 1][y], y) for y in states])
    return (prob, path[state])
 
def cut(sentence):
    #pdb.set_trace()
    prob, pos_list =  viterbi(sentence,('B','M','E','S'), prob_start, prob_trans, prob_emit)
    return (prob,pos_list)
 
if __name__ == "__main__":
    test_str = u"老师不只是简单的教书"
    prob,pos_list = cut(test_str)
    print  (test_str)
    print  (pos_list)
    test_str = u"还要教授学生处事的道理与主动学习的精神"
    prob,pos_list = cut(test_str)
    print  (test_str)
    print  (pos_list)
    test_str = u"一架从柏林飞往美国的客机悄无声息地降落在肯尼迪国际机场一个废弃的跑道上"
    prob,pos_list = cut(test_str)
    print  (test_str)
    print  (pos_list)
    test_str = u"权力的游戏"
    prob,pos_list = cut(test_str)
    print  (test_str)
    print  (pos_list)

老师不只是简单的教书
['B', 'E', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E']
还要教授学生处事的道理与主动学习的精神
['B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'S', 'B', 'E']
一架从柏林飞往美国的客机悄无声息地降落在肯尼迪国际机场一个废弃的跑道上
['B', 'E', 'S', 'B', 'E', 'B', 'E', 'B', 'E', 'S', 'S', 'B', 'E', 'S', 'B', 'E', 'B', 'E', 'B', 'E', 'B', 'M', 'S', 'B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S']
权力的游戏
['B', 'E', 'S', 'B', 'E']


In [1]:
import jieba.posseg as pseg
words =pseg.cut(u"我爱北京天安门")
for w in words:
    print(w.word,w.flag)
words =pseg.cut(u"一架从柏林飞往美国的客机悄无声息地降落在肯尼迪国际机场一个废弃的跑道上")
for w in words:
    print(w.word,w.flag) 


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\24463\AppData\Local\Temp\jieba.cache
Loading model cost 1.151 seconds.
Prefix dict has been built succesfully.


我 r
爱 v
北京 ns
天安门 ns
一架 m
从 p
柏林 nr
飞往 v
美国 ns
的 uj
客机 n
悄无声息 l
地 uv
降落 v
在 p
肯尼迪 nr
国际 n
机场 n
一个 m
废弃 v
的 uj
跑道 v
上 f


In [3]:
a = -0.26268660809250016
import math
math.exp(a)

0.7689828525554734

In [12]:
import jieba
import jieba.analyse
file_name = "userDic.txt"
jieba.load_userdict(file_name)
seg_list = jieba.cut("北风网", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式
sentence = "一架从柏林飞往美国的客机悄无声息地降落在肯尼迪国际机场一个废弃的跑道上，机上两百多个人员全部神秘死亡"
jieba.analyse.extract_tags(sentence, topK=5, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))

Default Mode: 北风网


['飞往', '客机', '跑道', '机上', '降落']